In [1]:
%matplotlib inline
from config_calc import *
import grid_tools

from collections import OrderedDict

import cesm_orcas_sci as cesm
import cam
import gv
import metpy.calc as mcalc

import matplotlib.pyplot as plt

### read model data set

In [ ]:
case = 'bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004'

tracer_def = cesm.trace_gas_tracers(case)
tracer_3d = [k for k in tracer_def]
varlist = ['Z3','Q','PS','PBLH','T','U','V','U10','PSL','OMEGA']+tracer_3d

dsa = cesm.open_casedata(case,'atm','cam.h0',varlist)
dsa.time

<xarray.DataArray 'time' (time: 3347)>
array(['2007-01-01T00:00:00.000000000', '2007-01-02T00:00:00.000000000',
       '2007-01-03T00:00:00.000000000', ..., '2016-02-27T00:00:00.000000000',
       '2016-02-28T00:00:00.000000000', '2016-02-29T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01 2007-01-02 2007-01-03 ...
Attributes:
    long_name:  time
    bounds:     time_bnds

### read ORCAS flight data
Find the right day in the model data and subset; store a list of obs datasets 

In [ ]:
RF = []
MDL = []
for f in gv.flight_file:
    rf = xr.open_dataset(f)
    RF.append(rf)
    MDL.append(dsa.sel(time=rf.time[0].astype('M8[D]'),method='nearest'))
dss = xr.concat(MDL,dim='time')

### compute pressure from hybrid levels

In [ ]:
%%time
dss = cam.pres_hybrid(dss,layer_center=True)
varlist = varlist+['Pm']

In [ ]:
reload(cam)
dss['theta'] = cam.potential_temperature(pressure=dss.Pm,
                                        temperature=dss['T'])
varlist = varlist+['theta']
dss

### convert units of tracer consituents

In [ ]:
ds = cesm.convert_dataset(dss,case)
ds = ds.compute()
print ds

### interpolate in xy to get a collection of model columns

In [ ]:
%%time
model_colums = []
for i,rf in enumerate(RF):
    var = [ds[v].isel(time=i) for v in varlist]
    model_colums.append(cam.interp_columns_esmf(rf.GGLON,rf.GGLAT,ds.lon,ds.lat,*var))
print model_colums[0]

### interpolate within the column to get point of measurement

In [ ]:
%%time
rf_model = []
for i,rf in enumerate(RF):
    var = [model_colums[i][v] for v in varlist]
    rf_model.append(cam.interp_within_column(rf.PSXC,model_colums[i].Pm,*var,
                                             model_z_surface=model_colums[i].PS))
        
print rf_model[0]

In [ ]:
copy_flight_var_list = ['Year','flt','DOY','UTC','n_prof','GGALT']
for i,rf in enumerate(RF):
    for v in copy_flight_var_list:
        rf_model[i][v] = rf[v]

In [ ]:
dir_out = os.path.join(os.path.dirname(gv.flight_file[0]),'..','cesm_flight_data')
if not os.path.exists(dir_out):
    call(['mkdir','-p',dir_out])
    
for i,rf in enumerate(rf_model):
    file_out = '.'.join([case,os.path.basename(gv.flight_file[i])])
    file_out = os.path.join(dir_out,file_out)
    print(file_out)
    rf.to_netcdf(file_out,unlimited_dims='time')